# Multicolinearidade

Neste segundo notebook sobre o assunto "Colinearidade", trataremos de um conceito relacionado, mas diferente da correlação linear.

Diferente da correlação linear, que envolve duas variáveis e pode ser medida usando parâmetros como R², a Multicolinearidade ocorre quando *duas ou mais variáveis independentes* do seu modelo podem prever uma outra variável independente.

Vamos imaginar o seguinte exemplo: usamos as variáveis A, B, C e D para prever o valor Y. Utilizamos a análise de correlação linear entre todas as variáveis independentes (A, B, C, D) e elas não estão correlacionadas entre si. Dessa forma, não temos o problema de variáveis correlacionadas. Porém, a Multicolinearidade é mais sutil. Ela ocorre quando, por exemplo, o valor da variável D pode ser previsto com um modelo linear envolvendo as variáveis A, B e C. Como o valor de D está relacionado aos valores de A, B e C, incluir a variável D no nosso modelo seria "redundante".

Note aqui um ponto importante: a presença de Multicolinearidade não invalida seu modelo - as predições feitas por ele continuam sendo válidas. Porém, você terá um problema se tentar **interpretar** o seu modelo, pois a presença de variáveis multicorrelacionadas impacta o valor dos coeficientes do modelo (no nosso exemplo, não podemos interpretar os valores dos coeficientes A, B, C pois parte do erro das variáveis está presente no coeficiente D).

Como detectar a Multicolinearidade em nossas variáveis independentes? O método mais comumente utilizado é chamado de Fator de inflação de variância (em inglês, *Variance Inflation Factor*, ou VIF). 

Em resumo, o valor de VIF para uma variável é a razão entre a variância de um modelo que usa todas as variáveis disponíveis (no nosso exemplo, A, B, C, D) pela variância de um modelo que usa somente uma variável. Ou seja, podemos medir 4 VIFs, um para cada variável.

Você pode usar os seguintes valores como guia:

- Uma variável com VIF igual a 1 não apresenta multicolinearidade
- Uma variável com VIF entre 1 e 5 apresenta multicolinearidade "leve", e não necessariamente precisa ser removida
- Uma variável com VIF maior que 5 apresenta multicolinearidade, é melhor retirá-la

Importante: não remova todas as variáveis com VIF alto de uma vez! Isso porque, ao remover uma das variáveis, é possível que o VIF das outras diminua (caso a variável retirada seja relacionda às outras por multicolinearidade).

Felizmente, temos um módulo em Python que implementa o cálculo de VIFs: `statsmodels`. O módulo já está presente no Anaconda, mas, se precisar instalar, use: `conda install -c anaconda statsmodels`

Fontes:

- https://universidadedosdados.medium.com/multicollinearity-is-not-correlation-38014cbfc710
- https://en.wikipedia.org/wiki/Variance_inflation_factor

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Delaney_descriptors.csv", sep=";")
df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,Solubilidade_medida
0,10.253329,-1.701605,10.253329,0.486602,0.217518,457.432,430.216,457.158411,178,0,...,0,0,0,0,0,0,0,0,0,-0.77
1,11.724911,-0.145880,11.724911,0.145880,0.811283,201.225,190.137,201.078979,76,0,...,0,0,0,0,0,0,0,0,0,-3.30
2,10.020498,0.845090,10.020498,0.845090,0.343706,152.237,136.109,152.120115,62,0,...,0,0,0,0,0,0,0,0,0,-2.06
3,2.270278,1.301055,2.270278,1.301055,0.291526,278.354,264.242,278.109550,102,0,...,0,0,0,0,0,0,0,0,0,-7.87
4,2.041667,1.712963,2.041667,1.712963,0.448927,84.143,80.111,84.003371,26,0,...,0,0,0,0,0,0,1,0,0,-1.33


In [3]:
# Vamos selecionar apenas algumas colunas para possibilitar uma melhor visualização dos coeficientes
X = df[["MolWt", "ExactMolWt", "FractionCSP3", "MolLogP", "NumAromaticRings", "NumHAcceptors", 
        "NumHDonors", "NumRotatableBonds", "TPSA", "LabuteASA"]]
y = df.iloc[:, -1]  # resposta

In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# Criar dataframe com todos os valores de VIF 
vif_data = pd.DataFrame()
vif_data["Variável"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

            Variável            VIF
0              MolWt  334684.421360
1         ExactMolWt  345475.127239
2       FractionCSP3       2.904945
3            MolLogP      28.164898
4   NumAromaticRings       4.581787
5      NumHAcceptors      17.680917
6         NumHDonors       4.692997
7  NumRotatableBonds       3.488453
8               TPSA      20.663980
9          LabuteASA     216.542770


Como visto anteriormente, MolWt e ExactMolWt são altamente correlacionados, e isso reflete no valor de VIF. Vejamos como ficam os VIFs se removemos um deles. Também vou remover LabuteASA, que deu um valor bem alto.

In [5]:
X = df[["MolWt", "FractionCSP3", "MolLogP", "NumAromaticRings", "NumHAcceptors", 
        "NumHDonors", "NumRotatableBonds", "TPSA"]]

# Criar dataframe com todos os valores de VIF 
vif_data = pd.DataFrame()
vif_data["Variável"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

            Variável        VIF
0              MolWt  35.950919
1       FractionCSP3   2.730528
2            MolLogP  22.967090
3   NumAromaticRings   4.101287
4      NumHAcceptors  17.212095
5         NumHDonors   4.512728
6  NumRotatableBonds   3.258018
7               TPSA  19.560133


Ainda temos VIFs altos para algumas variáveis. Vamos remover a com VIF mais alto (MolWt) e repetir.

In [6]:
# Removendo MolWt
X = df[["FractionCSP3", "MolLogP", "NumAromaticRings", "NumHAcceptors", 
        "NumHDonors", "NumRotatableBonds", "TPSA"]]

# Criar dataframe com todos os valores de VIF 
vif_data = pd.DataFrame()
vif_data["Variável"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

            Variável        VIF
0       FractionCSP3   2.730342
1            MolLogP   4.947061
2   NumAromaticRings   3.967379
3      NumHAcceptors  13.595432
4         NumHDonors   4.120221
5  NumRotatableBonds   2.990588
6               TPSA  17.907283


In [7]:
# Removendo TPSA
X = df[["FractionCSP3", "MolLogP", "NumAromaticRings", "NumHAcceptors", 
        "NumHDonors", "NumRotatableBonds"]]

# Criar dataframe com todos os valores de VIF 
vif_data = pd.DataFrame()
vif_data["Variável"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

            Variável       VIF
0       FractionCSP3  2.728052
1            MolLogP  4.946379
2   NumAromaticRings  3.954743
3      NumHAcceptors  4.141143
4         NumHDonors  2.428575
5  NumRotatableBonds  2.985479


Note que removendo a variável TPSA diminuimos o VIF de outras variáveis que estavam com VIF > 10. Note que nenhuma variável é completamente independente das outras. De qualquer forma, podemos dizer que o problema da multicolinearidade foi diminuído, e podemos criar um modelo interpretável com mais segurança.